In [17]:
### import necessary packages
import torch

from torch_geometric.profile import benchmark
# from torch_geometric.testing import (
#     onlyFullTest,
#     onlyLinux,
#     withDevice,
#     withPackage,
# )
from torch_geometric.utils import scatter

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Test Gather Scatter

In [25]:
def gather_scatter(x, edge_index, reduce='sum'):
    row, col = edge_index
    x_j = x[row]
    return scatter(x_j, col, dim_size=x.size(0), reduce=reduce)

x = torch.randn(10, 16, device=device)
edge_index = torch.randint(0, x.size(0), (2, 40), device=device)
edge_weight = torch.rand(edge_index.size(1), device=device)
matrix = torch.randn(x.size(-1), x.size(-1), device=device)

expected = gather_scatter(x, edge_index)
compiled_op = torch.compile(gather_scatter)
out = compiled_op(x, edge_index)
assert torch.allclose(out, expected, atol=1e-6)


In [24]:
row, col = edge_index
x_j = x[row]
# scatter(x_j, col, dim_size=x.size(0), reduce="add")
x_j.shape


torch.Size([40])

In [33]:
row, col = edge_index
x_ij = torch.cat([x[col], x[row]], dim=-1)
output =  scatter(x_ij, col, dim_size=x.size(0), reduce="add")
output.shape

torch.Size([10, 32])

In [35]:
x_ij.shape

torch.Size([40, 32])

In [7]:
expected.shape

torch.Size([10, 16])

In [9]:
!pip install torch_scatter

Defaulting to user installation because normal site-packages is not writeable
  Using cached torch_scatter-2.1.2.tar.gz (108 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for torch_scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=504074 sha256=7a86d9de13813f0bec94d33e1ef93f5d451c808890f50a0369a0975e54803dce
  Stored in directory: /home/yuxuan/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
Successfully built torch_scatter


In [10]:
from torch_scatter import scatter

src = torch.randn(10, 6, 64)
index = torch.tensor([0, 1, 0, 1, 2, 1])

# Broadcasting in the first and last dim.
out = scatter(src, index, dim=1, reduce="sum")

print(out.size())

torch.Size([10, 3, 64])


In [38]:
from setuptools import setup, Extension
from torch.utils import cpp_extension

setup(name='lltm_cpp',
      ext_modules=[cpp_extension.CppExtension('lltm_cpp', ['lltm.cpp'])],
      cmdclass={'build_ext': cpp_extension.BuildExtension})

AttributeError: 'tuple' object has no attribute 'tb_frame'